# **Note: Run this on the GPU runtime, will take a lot of time otherwise. Click Edit and Copy to get a runnable notebook**
Only works if you're signed into Kaggle

# Load Stuff 
Run this to load the needed libraries and create the needed models and functions.

**Run this one!**

In [7]:
from datasets import load_dataset

ds = load_dataset("alturing/gutenberg-texts")

Generating train split:   0%|          | 0/2951 [00:00<?, ? examples/s]

In [25]:
print(ds['train'][0]['text'].replace('_','').replace('â','')[:100])

ï»¿The Project Gutenberg eBook of Memoirs of a griffin, by Francis John   Bellew    This eBook is fo


In [8]:
import torch
import re

embed_size = int(384*1.5)
block_size = 256
dropout = 0.2
n_layer = 9
n_head = 6*2
device = "cuda" if torch.cuda.is_available() else "cpu"

has_trained = False

print(device)

textdemo = ''.join([ds['train'][i]['text'].replace('_','').replace('â','') for i in range(int(len(ds['train'])/1000))])
text = ''.join([ds['train'][i]['text'].replace('_','').replace('â','') for i in range(int(len(ds['train'])))])

preprocessed = [a for a in text]
preprocesseddemo = [a for a in textdemo]

print(len(text))
print(preprocesseddemo[:100])

vocab = sorted(list(set(preprocessed)))

vocab = sorted(vocab)
vocab_size = len(vocab)

print(vocab_size)


def encode(x):
    encoded = []
    for i in x:
        if i in vocab:
            encoded.append(vocab.index(i))
    return encoded

decode = lambda x: ''.join([vocab[i] for i in x])

print(vocab)

print(decode(encode("Rubber Ducks are nice")))
tokenizeddemo = torch.tensor(encode(preprocesseddemo))
train_data = tokenizeddemo[:int(len(tokenizeddemo)*0.9)]
val_data = tokenizeddemo[int(len(tokenizeddemo)*0.9):]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (32,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb,yb = get_batch('train')

import torch
import torch.nn as nn
from torch.nn import functional as F

class trans_block(nn.Module):
    def __init__(self,embed_size,heads):
        super().__init__()
        head_size = embed_size // heads
        self.attention = Heads(heads,head_size)
        self.ff_layer = FF_Layer(embed_size)
        self.lnorm1 = nn.LayerNorm(embed_size)
        self.lnorm2 = nn.LayerNorm(embed_size)
    def forward(self,x):
        x = x + self.attention(self.lnorm1(x))
        x = x + self.ff_layer(self.lnorm2(x))
        return x

class Head(nn.Module):
    def __init__(self,headsize):
        super().__init__()
        self.key = nn.Linear(embed_size,headsize,bias=False)
        self.query = nn.Linear(embed_size,headsize,bias=False)
        self.value = nn.Linear(embed_size,headsize,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        Batches, Time, Channels = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * Channels**-0.5
        wei = wei.masked_fill(self.tril[:Time,:Time] == 0,float('-inf'))
        wei = F.softmax(wei,dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out

class Heads(nn.Module):
    def __init__(self,n_head,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for i in range(n_head)])
        self.projection = nn.Linear(embed_size, embed_size)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([head(x) for head in self.heads],dim=-1)
        out = self.dropout(self.projection(out))
        return out

class FF_Layer(nn.Module):
    def __init__(self,embed_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_size,4*embed_size),
            nn.ReLU(),
            nn.Linear(4*embed_size,embed_size),
            nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.net(x)
class BigramLM(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size,embed_size)
        self.position_embedding_table = nn.Embedding(block_size,embed_size)
        self.lm_head = nn.Linear(embed_size,vocab_size)
        self.blocks = nn.Sequential(*[trans_block(embed_size,heads = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(embed_size)
    def forward(self,idx,targets=None):
        Branch,Time = idx.shape

        token_embed = self.embedding_table(idx)
        position_embed = self.position_embedding_table(torch.arange(Time,device=device))
        added = token_embed + position_embed
        added = self.blocks(added)
        added = self.ln_f(added)
        logits = self.lm_head(added)

        if targets is None:
            loss = None
        else:
            Batch, Time, Channel = logits.shape
            logits = logits.view(Batch*Time,Channel)
            targets = targets.view(Batch*Time)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self, idx, max_tokens):
        for i in range(max_tokens):
            idx_condition = idx[:, -block_size:]
            logits, loss = self(idx_condition)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

cuda
920554153
['ï', '»', '¿', 'T', 'h', 'e', ' ', 'P', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'G', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'B', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'M', 'e', 'm', 'o', 'i', 'r', 's', ' ', 'o', 'f', ' ', 'a', ' ', 'g', 'r', 'i', 'f', 'f', 'i', 'n', ',', ' ', 'b', 'y', ' ', 'F', 'r', 'a', 'n', 'c', 'i', 's', ' ', 'J', 'o', 'h', 'n', ' ', ' ', ' ', 'B', 'e', 'l', 'l', 'e', 'w', ' ', ' ', ' ', ' ', 'T', 'h', 'i', 's', ' ', 'e', 'B', 'o', 'o', 'k', ' ', 'i', 's', ' ', 'f', 'o']
192
[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x80', 

In [28]:
otext = open("/kaggle/input/sherlock-casebook/YAY.txt").read()
opreprocessed = [a for a in otext]
print(len(opreprocessed))
print(opreprocessed[:100])
ovocab = sorted(list(set(opreprocessed)))

ovocab = sorted(ovocab)
ovocab_size = len(ovocab)

def encode(x):
    encoded = []
    for i in x:
        if i in vocab:
            encoded.append(vocab.index(i))
    return encoded

decode = lambda x: ''.join([vocab[i] for i in x])

print(ovocab[:10])
print(encode(opreprocessed)[:100])

print(decode(encode("Rubber Ducks are nice")))
otokenized = torch.tensor(encode(opreprocessed))
otrain_data = tokenized[:int(len(otokenized)*0.9)]
oval_data = tokenized[int(len(otokenized)*0.9):]

def oget_batch(split):
    odata = otrain_data if split == 'train' else oval_data
    ix = torch.randint(len(odata) - block_size, (32,))
    x = torch.stack([odata[i:i+block_size] for i in ix])
    y = torch.stack([odata[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb,yb = get_batch('train')


452378
['P', 'R', 'E', 'F', 'A', 'C', 'E', '\n', '\n', 'I', ' ', 'f', 'e', 'a', 'r', ' ', 't', 'h', 'a', 't', ' ', 'M', 'r', '.', ' ', 'S', 'h', 'e', 'r', 'l', 'o', 'c', 'k', ' ', 'H', 'o', 'l', 'm', 'e', 's', ' ', 'm', 'a', 'y', ' ', 'b', 'e', 'c', 'o', 'm', 'e', ' ', 'l', 'i', 'k', 'e', ' ', 'o', 'n', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'o', 's', 'e', ' ', 'p', 'o', 'p', 'u', 'l', 'a', 'r', '\n', 't', 'e', 'n', 'o', 'r', 's', ' ', 'w', 'h', 'o', ',', ' ', 'h', 'a', 'v', 'i', 'n', 'g', ' ', 'o', 'u', 't']
['\n', ' ', '!', '"', "'", '(', ')', '+', ',', '-']
[48, 50, 37, 38, 33, 35, 37, 41, 0, 69, 68, 64, 81, 0, 83, 71, 64, 83, 0, 45, 81, 14, 0, 51, 71, 68, 81, 75, 78, 66, 74, 0, 40, 78, 75, 76, 68, 82, 0, 76, 64, 88, 0, 65, 68, 66, 78, 76, 68, 0, 75, 72, 74, 68, 0, 78, 77, 68, 0, 78, 69, 0, 83, 71, 78, 82, 68, 0, 79, 78, 79, 84, 75, 64, 81, 83, 68, 77, 78, 81, 82, 0, 86, 71, 78, 12, 0, 71, 64, 85, 72, 77, 70, 0, 78, 84, 83, 75, 72, 85]
Rubber Ducks are nice


# **Load Model and Train**
No need to run these two, these are just here to show what the training process looked like 

Uncomment these if you want to train the model from start, but there's a pretrained model in the dataset so no need to

***TRAINING IS A 15 MINUTE PROCESS***


**Don't run**

In [ ]:
int(len(ds['train']))

In [27]:
modelsomething = BigramLM()
model = modelsomething.to(device)
model.load_state_dict(torch.load("/kaggle/working/coder.txt",map_location = torch.device(device)))
out = model(xb,yb)

idx = torch.zeros((1,1),dtype=torch.long,device = device)
print("Before training:")
print(decode(model.generate(idx,max_tokens = 100)[0].tolist()))
print("\n")
optimizer = torch.optim.AdamW(model.parameters(),lr = 3e-4)

preprocessed = []
text = ''

for n in range(int(len(ds['train'])/100)):
    print("TRAINING DATA PART: "+str(n))
    
    start = 100*n
    stop = 100*(n+1)
        
    text = ''.join([ds['train'][j]['text'].replace('_','').replace('â','') for j in range(start,stop)])
    preprocessed = [a for a in text]
    
    print(len(preprocessed))
    
    print("TOKENIZING DATA")

    tokenized = torch.tensor(encode(preprocessed))
    train_data = tokenized[:int(len(tokenized)*0.9)]
    val_data = tokenized[int(len(tokenized)*0.9):]
                
    print('\n DATA PREPPED, STARTING TRAINING\n')  
    
    for i in range(5000):
        xb,yb = get_batch('train')
        _,loss = model(xb,yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print("Epoch: " + str(i) + " Loss: " +str(loss.item()))
            
    print(loss.item())
    has_trained = True
    print(decode(model.generate(idx,max_tokens = 500)[0].tolist())) #DON'T RUN THIS ONE UNLESS YOU'VE RUN THE ONE ABOVE, THIS IS JUST A RANDOM PIECE OF THE MODEL'S OUTPUT
    torch.save(model.state_dict(),"/kaggle/working/coder.txt")


/tmp/ipykernel_36/1307007159.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/coder.txt",map_location = torch.device(dev

Before training:
 of Rickert Bengarby.    With all the Orian Pastorean A. Vocaby. For Vanzetti and the Stased;  of Car


TRAINING DATA PART: 0
38580114
TOKENIZING DATA

 DATA PREPPED, STARTING TRAINING

Epoch: 0 Loss: 1.4254297018051147
Epoch: 100 Loss: 1.4373050928115845
Epoch: 200 Loss: 1.3848265409469604
Epoch: 300 Loss: 1.4600664377212524
Epoch: 400 Loss: 1.35837984085083
Epoch: 500 Loss: 1.3456300497055054
Epoch: 600 Loss: 1.4000132083892822
Epoch: 700 Loss: 1.3558262586593628
Epoch: 800 Loss: 1.3661648035049438
Epoch: 900 Loss: 1.3252897262573242
Epoch: 1000 Loss: 1.3434067964553833
Epoch: 1100 Loss: 1.4138115644454956
Epoch: 1200 Loss: 1.277002215385437
Epoch: 1300 Loss: 1.3196065425872803
Epoch: 1400 Loss: 1.3705353736877441
Epoch: 1500 Loss: 1.3877301216125488
Epoch: 1600 Loss: 1.3279204368591309
Epoch: 1700 Loss: 1.2810215950012207
Epoch: 1800 Loss: 1.3881113529205322
Epoch: 1900 Loss: 1.3739713430404663
Epoch: 2000 Loss: 1.2795988321304321
Epoch: 2100 Loss: 1.375571727752685

KeyboardInterrupt: 

A loot of extra training
(Don't run either)

In [34]:
for i in range(500):
    xb,yb = oget_batch('train')
    _,loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print("Epoch: " + str(i) + " Loss: " +str(loss.item()))
print(loss.item())


Epoch: 0 Loss: 0.09694873541593552
Epoch: 100 Loss: 0.10164610296487808
Epoch: 200 Loss: 0.0937911793589592
Epoch: 300 Loss: 0.09970223158597946
Epoch: 400 Loss: 0.09518542140722275
0.09747062623500824


***Don't run these unless you've run the one above, the model is loaded in the cell below these two***

In [35]:
 print("After Training:")
 print(decode(model.generate(idx,max_tokens = 500)[0].tolist())) #DON'T RUN THIS ONE UNLESS YOU'VE RUN THE ONE ABOVE, THIS IS JUST A RANDOM PIECE OF THE MODEL'S OUTPUT

After Training:
 frame, yet  possessing thereby an aspect of leading upon the poet soldier,  and we looked at an embroidery frame.  The man, nevertheless, had ever  become really prolific; I had a feeling for colour and characteristic  originality, the masterful treatment of painting and sudden success, and  had ascure success to the modern artists who attained to great art  more than any art, should be ascribed by painting as the statue of  composition.  In America, Mozart's preceding  operas, which was the sam


In [2]:
model = torch.jit.load('/kaggle/working/state.txt')
model.eval()

RuntimeError: PytorchStreamReader failed locating file constants.pkl: file not found

# **Loading the model**
Run these two to load and run the model!

**Definitely Run!**

In [ ]:
print(device)
modelsomething2 = BigramLM()
model2 = modelsomething2.to(device)
print(has_trained)
if has_trained == False:
    model2.load_state_dict(torch.load("/kaggle/input/sherlock-casebook/finalmodpre(1).txt",map_location = torch.device(device)))
else:
    model2.load_state_dict(torch.load("/kaggle/working/state.txt",map_location = torch.device(device)))
print("Run successfully")

**Run this and enter your prompt!**

In [1]:
import torch
model = torch.load("/kaggle/working/state.txt",map_location = torch.device('cuda'))
model.eval()

/tmp/ipykernel_37/1312102747.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/kaggle/working/state.txt",map_location = torch.device('cuda'))


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
text = open('/kaggle/input/novels/YAY.txt').read()

In [36]:
while True:
  contextc = input("Type your prompt/context! (type EXIT to stop)")
  print("\n")
  contextc = torch.tensor([encode(contextc)])
  context = contextc.to(device)
  if(decode(context[0].tolist()) == "EXIT"):
    break
  print(decode(model.generate(context,max_tokens = 500)[0].tolist()))
  #CHANGE MAX_TOKENS FOR DIFFERENT LENGTH
  print("\n")

Type your prompt/context! (type EXIT to stop) Hullo, said the mysterious man




Hullo, said the mysterious manner:      "Thick, a black barge of Art unfinished, is one of Voney Leopold Mozart's best  songs remind one of Mozart's father's principal works.  The compositions of  Mozart's great devotion were to have it had "exoradements" adhered  from this style to say many masters of old quartetts, some of them from the  Cross, of which John he made with the Working Men (copy.    Nevertheless, 190.    Second Giotto (unfinished state of Venice), Michelangelo and Brussels.      Before the first number of the




Type your prompt/context! (type EXIT to stop) Sherlock Holmes




Sherlock Holmes.          The Day Dream The Stadt-Theatre (water-colour).  Rae Collection.           The Brown Tombs (water-colour).  Rae Collection.          Wedding of St. George (water-colour).  S. Pepys Cockerell.           Joan of Art Gallery.           Crucifixion (Mort Art Gallery.)    1862.  St. George Merciles.           Morris and Brown (cartoons for Morris windows).             Christmas Caroldore (oil).  Mrs. Rossetti).    Charles Butler.           The Sea Spell (oil).          Proserpine (oil).  C




Type your prompt/context! (type EXIT to stop) EXIT


In [3]:
import torch
eh = torch.load("/kaggle/working/coder.txt",map_location = torch.device('cuda'))

/tmp/ipykernel_36/1526828279.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eh = torch.load("/kaggle/working/coder.txt",map_location = torch.device('cuda'))


In [5]:
eh(torch.tensor([1,2,3,5]))

TypeError: 'collections.OrderedDict' object is not callable